# Pytorch Workflow

Lets explore an example of the PyTorch end-to-end workflow.

In [1]:
what_were_covering = {1: "Get Data Ready", 2: "build model", 3: "fitting the model to the data (training)", 4: "making predictions and evaluating model", 5: "saving and loading a model", 6: "putting it all together"}
what_were_covering

{1: 'Get Data Ready',
 2: 'build model',
 3: 'fitting the model to the data (training)',
 4: 'making predictions and evaluating model',
 5: 'saving and loading a model',
 6: 'putting it all together'}

In [2]:
import torch
from torch import nn ## nn contains all of PyTorches building blocks for neural networks
import matplotlib.pyplot as plt

# cvheck PyTorch version
torch.__version__

'1.13.0'

## 1. Data (preparing and loading)

Data can be almost anything ... in machine learning.

* Excel spreadsheet
* Images of any kind
* Videos (youtube has lots of data)
* Audio like songs or podcasts
* DNA
* Text

Machine learning is a game of two parts:
1. Get data into a numerical representation.
2. Build a model to learn paters in that numerical representation. 

To showcase this, lets create some *known* data using the linear regression formula.

we'll use a linear regression formula to make a straight line with known parameters.

In [3]:
# Create known parameters

weight = 0.7
bias = 0.3

# Create 
start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

X[:10], y[:10], len(X), len(y)

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]),
 50,
 50)

### Splitting data into training and sets ( one of the most important concepts in machine learning in general)

Let's create a training and test set with our data.

In [5]:
# Create a train/test split
train_split = int(0.8 * len(X))
X_train, y_train = X[: train_split], y[: train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)